# COMP9318 Lab2

## Instructions
1. This note book contains instructions for **COMP9318-Lab2**.

* You are required to complete your implementation in a file `submission.py` provided along with this notebook.

* You are only allowed to use **Python 3.6** for implementation.

* You are not allowed to print out unnecessary stuff. We will not consider any output printed out on the screen. All results should be returned in appropriate data structures return by corresponding functions.

* You need to submit the code for **Lab2** via following link: https://kg.cse.unsw.edu.au/submit/

* For each question, we have provided you with detailed instructions along with question headings. In case of any problem, you can post your query @ Ed.

* If you choose to skip a question, leave the corresponding function body as it is (i.e., keep the `pass` line), otherwise it may affect your mark for other questions.

* You are allowed to add other functions and/or import additional modules (you may have to in this lab), but you are not allowed to define global variables. **Only functions are allowed** in `submission.py`. 

* You should not import unnecessary modules/libraries, failing to import such modules at test time will lead to errors.

* We will provide immediate feedback on your submission. You can access your scores using the online submission portal on the same day. 

* For **Final Evaluation** we will be using a different dataset, so your final scores may vary.  

* You are allowed to submit as many times as you want before the deadline, but **ONLY the latest version will be kept and marked**.

* Submission deadline for this assignment is **20:59:59 on 18th March, 2021 (Sydney Time)**. We will **not** accept any late submissions.

# Question 1: Optimized BUC algorithm (100 points)

You need to implement the full `buc_rec_optimized` algorithm with the single-tuple optimization (as described below). Given an input dataframe:

 A | B | M 
---|---|---
 1 | 2 | 100
 2 | 1 | 20

Invoking  `buc_rec_optimized` on this data will result in following dataframe: 


 A | B | M
---|---|---
 1 | 2 | 100
 1 |ALL| 100
 2 | 1 | 20 
 2 |ALL| 20
ALL| 1 | 20
ALL| 2 | 100
ALL|ALL| 120

We have pre-defined the function `buc_rec_optimized` in the file `submission.py`, and its helper functions are defined in the file `helper.py`. 

**Note:** You should use the functions defined in the file `helper.py`, you are not allowed to change this file. We will provide this file in the test environment.

## Input and output

Both `input` and `output` are dataframes.

The input dataframe (i.e., the base cuboid) is directly generated from the input file. Given the dimensionality of the base cuboid is $d$, each row is like:

<pre>
v_1  v_2 ...  v_d  m
</pre>

where v_i is the cell's value on the i-th dimension, and m is the measure value. 

The output dataframe contains $n$ rows, each for a non-empty cell in the compute data cube derived from the input base cuboid. Each row is formatted like input:

<pre>
v_1  v_2 ...  v_d  m
</pre>

where v_i is the cell's value on the i-th dimension, and m is the measure value. 


## The single-tuple optimization

Consider the naive way of recursive implementation of the BUC algorithm, you will notice that it uses several recursive calls to compute all the derived results from an input that consists of only one tuple. This is certainly a waste of computation. 

For example, if we are asked to compute the cube given the following input

 B | C | M 
---|---|---
 1 | 2 | 100

We can immmediately output the following, **without** using any recursive calls. 

<pre>
1    2    100
*    2    100
1    *    100
*    *    100
</pre>



** Note: For lab-2, you are allowed to use only two libraries, i.e., pandas, and numpy.** 

In [19]:
import pandas as pd
import numpy as np

In [20]:
##============================================================
# Data file format: 
# * tab-delimited input file
# * 1st line: dimension names and the last dimension is assumed to be the measure
# * rest of the lines: data values.

def read_data(filename):
    df = pd.read_csv(filename)
    return (df)
      
# helper functions
def project_data(df, d):
    # Return only the d-th column of INPUT
    return df.iloc[:, d]

def select_data(df, d, val):
    # SELECT * FROM INPUT WHERE input.d = val
    col_name = df.columns[d]
    return df[df[col_name] == val]

def remove_first_dim(df):
    # Remove the first dim of the input
    return df.iloc[:, 1:]

def slice_data_dim0(df, v):
    # syntactic sugar to get R_{ALL} in a less verbose way
    df_temp = select_data(df, 0, v)
    return remove_first_dim(df_temp)

In [27]:
def checkAllInArray(array1, arrayOrg):
    
    arrayLen = len(array1) - 1
    while arrayLen >= 0:
        if array1[arrayLen] != 'ALL':
            array1[arrayLen] = 'ALL'
            return True
        else:
            array1[arrayLen] = arrayOrg[arrayLen]

        arrayLen -= 1

    return False



def singleTupleOptim(df):
    
    
    #dColumn = project_data(df, )
    value = df.iloc[0,-1]
    dimensionLessArray = df.iloc[0, : -1]
    arrayOrg = np.copy(dimensionLessArray.values)
    array1 = np.zeros(df.values.shape[1] - 1, dtype = object)

    for i in range(len(array1)):
        array1[i] = arrayOrg[i]

    returnValue = []
    returnValue.append(df.values[0])

    while checkAllInArray(array1, arrayOrg):       
        
        rowArray = np.zeros(df.values.shape[1], dtype = object)

        for j in range(len(array1)):
            rowArray[j] = array1[j]
        print(rowArray)
        rowArray[-1] = value
        returnValue.append(rowArray)
        
    return returnValue


def buc_rec_optimized(df):
    
    emptyDataFrame = pd.DataFrame(columns = df.columns)
    dictionary = {}
    
    for i in range(df.values.shape[0]):
        rows = singleTupleOptim(df.iloc[i:i+1, :])
        print(1)
        for element in rows:
            value = element[-1]
            key = ""

            for j in range(len(element) - 1):
                key += str(element[j])
                if j < len(element) - 2:
                    key += '~'
            
            if key in dictionary:
                dictionary[key] += (value)
            else:
                dictionary[key] = (value)

    index = 0
    for key, value in sorted(dictionary.items()):

        listString = key.split('~')
        newElementForRow = np.zeros((len(listString) + 1), dtype = object)
        
        for i in range(len(listString)):
                newElementForRow[i] = listString[i]

        newElementForRow[-1] = value

        emptyDataFrame.loc[index] = newElementForRow
        index += 1
            
    return emptyDataFrame

In [28]:
## You can test your implementation using the following code...
import helper
import submission as submission
input_data = read_data('./asset/helper.csv')
input_data

,A,B,C,M
0,1,1,2,10
1,1,2,1,20
2,2,1,1,30


In [29]:
output = buc_rec_optimized(input_data)
output

[1 1 'ALL' 0]
[1 'ALL' 2 0]
[1 'ALL' 'ALL' 0]
['ALL' 1 2 0]
['ALL' 1 'ALL' 0]
['ALL' 'ALL' 2 0]
['ALL' 'ALL' 'ALL' 0]
1
[1 2 'ALL' 0]
[1 'ALL' 1 0]
[1 'ALL' 'ALL' 0]
['ALL' 2 1 0]
['ALL' 2 'ALL' 0]
['ALL' 'ALL' 1 0]
['ALL' 'ALL' 'ALL' 0]
1
[2 1 'ALL' 0]
[2 'ALL' 1 0]
[2 'ALL' 'ALL' 0]
['ALL' 1 1 0]
['ALL' 1 'ALL' 0]
['ALL' 'ALL' 1 0]
['ALL' 'ALL' 'ALL' 0]
1


,A,B,C,M
0,1,1,2,10
1,1,1,ALL,10
2,1,2,1,20
3,1,2,ALL,20
4,1,ALL,1,20
5,1,ALL,2,10
6,1,ALL,ALL,30
7,2,1,1,30
8,2,1,ALL,30
9,2,ALL,1,30


,A,B,M
0,2,1,20
1,2,ALL,20
2,3,4,100
3,3,ALL,100
4,ALL,1,20
5,ALL,4,100
6,ALL,ALL,120
